# Lineare Diskriminanzanalyse
## Araz, Hasenklever, Pede

Laden von Bibliotheken

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import pandas as pd
import random
import scipy

from scipy.stats import zscore

# interactive
from ipywidgets.widgets import interact, IntSlider, FloatSlider
from IPython.display import display


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from multiDatenanalyse import *

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

mmPfad = r"D:\C\Uni\Master\KSS\MV_Analyse\Messmatrix.csv"#'../data/Messmatrix.csv'

## Laden der Merkmalsmatrix
### und Vorverarbeitung von Zeilen und Spalten nach Anzahl von NaNs und stark korrelierenden Merkmalen

In [ ]:
df = load_data(mmPfad)

## Produktweise Sortierung der Daten

minimale Anzahl der Rohre je Walzlos festlegen
und Daten nach Produkten sortieren

In [ ]:
min_num_walzlos = 300
df_all_prod = [extract_product(df, product_id=product_id, min_num_walzlos=min_num_walzlos) for product_id in range(26)]

Auswahl des Produtes durch Schieberegler implementieren

TODO Produkteigenschaften (LG, AD, WD) ausgeben

In [ ]:
@interact(index=IntSlider(min=0, max=26, value = 11))
def count_per_product(index):
    print("Anzahl der Walzlose: "+str(len(pd.unique(df_all_prod[index]["Header_Walzlos"]))))

Auswahl eines Produktes und Ausgabe der Anzahl von Walzlosen mit "genügend" Rohren

In [ ]:
product_id = 11
df_prod = df_all_prod[product_id]
print("Anzahl der Walzlose: "+str(len(pd.unique(df_prod["Header_Walzlos"]))))

Verbleibende Merkmale:

In [ ]:
df_prod.columns

## Aufteilen der Daten in Test- und Trainingsdaten

In [ ]:
test_frac = 0.4
train_set, test_set = get_lda_data(df_prod, test_frac=test_frac)

## Normalisierung der Daten

In [ ]:
train_set['data'] = zscore(train_set['data'])
test_set['data'] = zscore(test_set['data'])

## Kovarianzmatrix von Trainings- und Testdaten

In [ ]:
cov_train = np.cov(train_set['data'].T)
cov_test = np.cov(test_set['data'].T)

plt.figure(figsize=(15,10))
ax1 = plt.subplot(121)
ax1.imshow(255*(cov_train-np.max(cov_train))/(np.max(cov_train)-np.min(cov_train)), 'gray')
ax1.set_title('Kovarianz der Trainingsdaten')
ax1.set_xlabel('Merkmal')
ax1.set_ylabel('Merkmal')

ax2 = plt.subplot(122)
ax2.imshow(255*(cov_test-np.max(cov_test))/(np.max(cov_test)-np.min(cov_test)), 'gray')
ax2.set_title('Kovarianz der Testdaten')
ax2.set_xlabel('Merkmal')
ax2.set_ylabel('Merkmal')
print('Wie selbstähnlich sind die Test- und Trainingsdaten?')

## Dürchführen der LDA auf die Trainingsdaten

In [ ]:
# extract data and label
X_train, y_train = train_set['data'], train_set['label']
X_test, y_test = test_set['data'], test_set['label']

# number components for transform
n_components = 3

# LDA object
sklearn_LDA = LDA(n_components=n_components, solver='eigen')

# fit with train data
sklearn_LDA = sklearn_LDA.fit(X_train, y_train)

## Darstellung der Eigenwerte

In [ ]:
plt.stem(sklearn_LDA.explained_variance_ratio_)
plt.xlabel('Index Eigenwert')
plt.ylabel('Beitrag zur Varianz')
plt.title("Eigenwerte")

## Testen der Klassifikation

In [ ]:
train_pred = sklearn_LDA.predict(X_train)
print('{0:.2f}% Genauigkeit bei der Klassifikation der Trainingsdaten'.format(100*np.mean(train_pred == y_train)))

test_pred = sklearn_LDA.predict(X_test)
print('{0:.2f}% Genauigkeit bei der Klassifikation der Testdaten'.format(100*np.mean(test_pred == y_test)))

## Darstellung der transformierten Trainingsdaten und Klassenzugehörigkeit

In [ ]:
data = sklearn_LDA.transform(X_train)
plot_lda(data, y_train, 'Transformierte Trainingsdaten')

## Interpretation der LDA-Ergebnisse

### Die Helligkeit der Punkte bildet die Größe des Beitrags des Merkmals im jeweiligen Eigenvektor ab. 

In [ ]:
eigvecs = sklearn_LDA.scalings_
plt.figure(figsize=(20,5))
plt.imshow(np.abs(eigvecs), 'gray')
#_ = plt.axis('off')
plt.title("Eigenvektoren")

In [ ]:
print('Einflussreichstes Merkmal im ersten EV: {}'.format(df[df.columns[6:]].columns[np.argmax(np.abs(eigvecs[:, 0]))]))
print('Einflussreichstes Merkmal im zweiten EV: {}'.format(df[df.columns[6:]].columns[np.argmax(np.abs(eigvecs[:, 1]))]))

## Darstellung der Eigenvektoren

In [ ]:
plt.figure(figsize=(20,10))

for index in range(3):
    ax = plt.subplot(1,3,index+1)
    ax.stem(eigvecs[:, index])
    ax.set_title('Eigenvektor {}'.format(index))
    ax.set_xlabel('Merkmalsindex')
    ax.set_ylabel('Beitrag in Eigenvektor')